# Диспетчеризация

**Диспетчеризация** -- выбор реализации полиморфной функции для выполнения операции над заданными типами операндов.

Иногда диспетчеризацию отличают от **перегрузки** -- ситуации, когда выбор производится на этапе компиляции.

## Полиморфизм

Под полиморфизмом понимается ситуация, когда один и тот же программный код соответствует различному машинному коду, в зависимости от контекста.

**Пример 1**

В Си полиморфизма практически нет, но арифметические операторы всё же полиморфные.

Полиморфное поведение арифметических операторов имеет место практически во всех высокоуровневых ЯП.

In [1]:
@code_native 2 + 4

	.text
; ┌ @ int.jl:87 within `+`
	leaq	(%rdi,%rsi), %rax
	retq
	nopw	%cs:(%rax,%rax)
	nop
; └


In [2]:
@code_native 2.0 + 4.0

	.text
; ┌ @ float.jl:399 within `+`
	vaddsd	%xmm1, %xmm0, %xmm0
	retq
	nopw	%cs:(%rax,%rax)
	nop
; └


Отсутствие полиморфизма ведёт к тому, что одинаковые по смыслу операции для разных типов данных нужно обозначать разными символами (именами): например, в Си `abs(n)` работает только с целыми числами, `fabs(x)` -- с числами в двойной точности.

**Пример 2**

В Python функция `len` -- полиморфна, аргументами могут быть объекты разнообразных типов -- кортежи, массивы, словари и т.д. Такое поведение, очевидно, упрощает написание более сложных функций, которые работают с разными типами данных.

## Перегрузка функций

C++, например, позволяет определить несколько функций с одним и тем же именем, работающих с разными типами данных. Функцию взятия модуля, например, можно определить так:
```cpp
struct complex
{
    double rpart;
    double ipart;
};

int abs(int n)
{
    return (n > 0) ? n : (-n);
}

double abs(double x)
{
    return (x > 0) ? x : (-x);
}

complex abs(complex z)
{
    return sqrt(z.rpart * z.rpart + z.ipart * z.ipart);
}
```

Перегрузка реализует *статический полиморфизм* -- выбор конкретной реализации делается на этапе компиляции, до выполнения программы.

## Динамический полиморфизм

Одна из мотиваций для создания ООП -- возможность представить в памяти компьютера одну и ту же абстракцию разными способами. Чтобы "клиент" -- программист, использующий реализацию, -- имел возможность использовать любую из альтернативных реализаций как чёрный ящик без переписывания кода, нужно:

1. дать возможность присвоить всем реализациям один тип
2. выбирать реализацию процедур на этапе выполнения программы (т.к. на этапе компиляции аргумент имеет один и тот же "абстрактный" тип)

## Одинарная диспетчеризация

В ООП динамический полиморфизм реализуется на основе *диспетчеризации по первому аргументу*, при этом вместо записи `foo(x, y, z)` используется `x.foo(y, z)`. Исторически такое представление связано с простотой реализации в рамках языков с ранним связыванием через таблицу виртуальных методов.

Одна из возникающих проблем -- это реализация методов, поведение которых должно зависеть от типов нескольких объектов.

**Пример**

```python
class ComplexNumber:
    pass

class ComplexCart(ComplexNumber):
    def __init__(self, x, y=0):
        self._re = x
        self._im = y
        
    def add(self, z):
        return ComplexCart(self._re + z.realpart(), self._im + z.imagpart())
    
    def realpart(self):
        return self._re
    
    def imagpart(self):
        return self._im

class ComplexPolar(ComplexNumber):
    def __init__(self, r, phi=0):
        self._r = r
        self._phi = phi
        
    def add(self, z):
        re = self.realpart() + z.realpart()
        im = self.imagpart() + z.imagpart()
        return ComplexPolar(sqrt(re**2 + im**2), atan2(im, re))
    
    def realpart(self):
        return self._r * cos(self._phi)
    
    def imagpart(self):
        return self._r * sin(self._phi)
```

Здесь возникает две проблемы:
1. Процедура `add` работает только внутри класса комплексных чисел (т.е. нельзя сложить комплексное число с целым).
2. Результат работы `add` зависит от объекта, который поставлен первым аргументом -- т.е. сложение, в некотором смысле, будет некоммутативным.

До некоторой степени можно исправить первый недостаток, подправив инициализацию. Например:

```python
class ComplexPolar(ComplexNumber):
    def __init__(self, x, phi=0):
        if isinstance(x, ComplexNumber):
            r = sqrt(x.realpart()**2 + x.imagpart()**2)
            phi = atan2(x.imagpart(), x.realpart())
        else:
            r = x
        self._r = r
        self._phi = phi
    
    def add(self, z):
        z = ComplexPolar(z)
        ...
```
Однако это оставляет другую проблему -- если появляется другой класс, для которого хотелось бы определить процедуру сложения с комплексным числом, мы должны добавить к `ComplexCart` и `ComplexPolar` процедуру инициализации с объектом нового типа.

## Множественная диспетчеризация

При множественной диспетчеризации выбор нужного метода идёт на основе типов всех аргументов, а не только одного выделенного.

Рассмотрим пример: реализация комплексных чисел в декартовом и полярном представлениях (типы `ComplexCart` и `ComplexPolar`).

Необходимо реализовать процедуры сложения и умножения чисел со следующими свойствами:
* сложение и умножение чисел одинакового типа возвращает тот же тип
* сложение и умножение комплексного числа и действительного даёт тип комплексного числа
* для разных комплексных типов будем считать, что нам предпочтителен тип результата `ComplexCart`

#### Типы данных

In [3]:
abstract type ComplexNumber end

struct ComplexCart <: ComplexNumber
    _re::Float64
    _im::Float64
end

struct ComplexPolar <: ComplexNumber
    _r::Float64
    _φ::Float64
end

#### Вспомогательные функии для получения действительной и мнимой частей

In [4]:
function realpart(z::ComplexCart)
    return z._re
end

function imagpart(z::ComplexCart)
    return z._im
end

function realpart(z::ComplexPolar)
    return z._r * cos(z._φ)
end

function imagpart(z::ComplexPolar)
    return z._r * sin(z._φ)
end

imagpart (generic function with 2 methods)

#### Сложение двух комплексных чисел в общем случае

In [5]:
function Base.:+(z1::ComplexNumber, z2::ComplexNumber)
    return ComplexCart(realpart(z1) + realpart(z2), imagpart(z1) + imagpart(z2))
end

За счет диспетчеризации в функциях `realpart` и `imagpart` функция работает как со слагаемыми одного типа, так и разных типов:

In [6]:
ComplexCart(4, 5) + ComplexCart(3, 3)

ComplexCart(7.0, 8.0)

In [7]:
ComplexCart(3, 4) + ComplexPolar(5, atan(4, 3))

ComplexCart(6.0, 8.0)

In [8]:
ComplexPolar(5, atan(4, 3)) + ComplexCart(3, 4)

ComplexCart(6.0, 8.0)

Но пока что не работает сложение чисел в полярном представлении так, как хотелось бы.

In [9]:
ComplexPolar(5, atan(4, 3)) + ComplexPolar(5, atan(4, 3))

ComplexCart(6.000000000000001, 7.999999999999999)

#### Добавление отдельного метода для полярного представления

In [10]:
function Base.:+(z1::ComplexPolar, z2::ComplexPolar)
    re = realpart(z1) + realpart(z2)
    im = imagpart(z1) + imagpart(z2)
    return ComplexPolar(hypot(re, im), atan(im, re))
end

In [11]:
ComplexPolar(5, atan(4, 3)) + ComplexPolar(5, atan(4, 3))

ComplexPolar(10.0, 0.9272952180016121)

#### Реализация операции между комплексным числом и действительным

In [12]:
function Base.:+(z::ComplexCart, x::Real)
    return ComplexCart(realpart(z) + x, imagpart(z))
end

Base.:+(x::Real, z::ComplexCart) = z + x

In [13]:
ComplexCart(3, 4) + 7

ComplexCart(10.0, 4.0)

In [14]:
7 + ComplexCart(3, 4)

ComplexCart(10.0, 4.0)

Можно записать реализацию операции между комплексным и действительным числом в более общем виде, определив конструкторы комплексных чисел из действительных. Тогда для сложения мы сначала преобразуем действительное число в комплексное нужного типа, а затем складываем два аргумента уже как комплексные числа:

In [15]:
function Base.:+(z::ComplexNumber, x::Real)
    x_cplx = typeof(z)(x)
    return z + x_cplx
end

Base.:+(x::Real, z::ComplexNumber) = z + x

In [16]:
ComplexCart(x::Real) = ComplexCart(x, 0)

ComplexPolar(x::Real) = ComplexPolar(abs(x), x >= 0 ? 0.0 : pi)

ComplexPolar

In [17]:
ComplexPolar(4, pi/2) + 3

ComplexPolar(5.0, 0.9272952180016121)

Обратим внимание, что конкретная вызываемая операция `+` зависит от *типов обоих слагаемых*, поэтому схема одиночной диспетчеризации, принятая в ООП, здесь не подходит в полной мере.

При помощи же множественной диспетчеризации мы можем написать свой метод под любую комбинацию типов операндов.

При необходимости, если будет добавлен ещё один тип представления комплексных чисел, нужные методы легко добавляются.

В традиционном ООП выбор метода на основе типов двух аргументов возможно реализовать с помощью паттерна "двойная диспетчеризация", но для трёх и более аргументов решения нет. В случае же простой перегрузки функций в C++ не реализуется динамический выбор реализации.

### Пример паттерна диспетчеризации -- объявление функции для работы с потоком ввода-вывода или с именем файла

Задача: получить номер шага из дампа LAMMPS

```
ITEM: TIMESTEP
1800
ITEM: NUMBER OF ATOMS
1000
ITEM: BOX BOUNDS pp pp pp
0.0000000000000000e+00 1.1856311014966876e+01
0.0000000000000000e+00 1.1856311014966876e+01
0.0000000000000000e+00 1.1856311014966876e+01
...
```

Сначала определим метод, который принимает на вход поток ввода-вывода, читает оттуда данные и парсит их:

In [18]:
function parse_lammps_dump(io::IO)
    ln = readline(io)
    startswith(ln, "ITEM: TIMESTEP") || error()
    
    tstep_str = readline(io)
    return parse(Int, tstep_str)
end

parse_lammps_dump (generic function with 1 method)

Теперь определим метод с таким же именем, но строковым аргументом. Строка интерпретируется как имя файла, который открывается, и работа с полученным файловым потоком делегируется ранее введённому методу.

In [19]:
function parse_lammps_dump(fname::AbstractString)
    open(fname) do io
        return parse_lammps_dump(io)
    end
end

parse_lammps_dump (generic function with 2 methods)

In [20]:
parse_lammps_dump(
    IOBuffer("""
    ITEM: TIMESTEP
    1800
    ITEM: NUMBER OF ATOMS
    1000
    ITEM: BOX BOUNDS pp pp pp
    0.0000000000000000e+00 1.1856311014966876e+01
    0.0000000000000000e+00 1.1856311014966876e+01
    0.0000000000000000e+00 1.1856311014966876e+01
    """)
)

1800

In [21]:
parse_lammps_dump("/home/vvp/Work/Simulations/test/ljstat/lj.0.75.1000.dump")

1000